In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Load CIFAR-10 dataset from keras
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
class_labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
print('Dataset class labels', class_labels)

# Explore the dataset
# Split the training set into training and validation sets, validation set will be 20% of training set
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
print("\n")
print('Training set input shape', x_train.shape)
print('Training set target shape', y_train.shape)
print('Validation set input shape', x_val.shape)
print('Validation set target shape', y_val.shape)
print('Test set input shape', x_test.shape)
print('Test set target shape', y_test.shape)
print("\n")

# One-hot encoding, it will help in training stage since we have categorical data
y_train_one_hot = keras.utils.to_categorical(y_train, 10)
y_val_one_hot = keras.utils.to_categorical(y_val, 10)
y_test_one_hot = keras.utils.to_categorical(y_test, 10)
print('Train one hot shape',y_train_one_hot.shape)
print('Validation one hot shape',y_val_one_hot.shape)
print('Test one hot shape',y_test_one_hot.shape)
print("\n")

print('Example of one hot label and its corresponding image')
%matplotlib inline
print('Label for the second image is: ', np.array(class_labels)[y_train[1].astype(int)]) # Convert the class_labels list into an array and then display the label
print('The one hot label for the second image is: ', y_train_one_hot[1])
img = plt.imshow(x_train[1])


# Image processing
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_val = x_val/255
x_test = x_test/255

# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train_one_hot, batch_size=64)

In [ ]:
# Custom CNN implementation
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#hist = model.fit(x_train, y_train_one_hot, batch_size=32, epochs=100, validation_split=0.2)
steps = int(x_train.shape[0] / 64)
hist = model.fit(it_train, steps_per_epoch=steps, epochs=100, validation_data=(x_val, y_val_one_hot))

In [ ]:
# Plot model loss and accuracy
plt.plot(hist.history['loss']) 
plt.plot(hist.history['val_loss']) 
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right') 
plt.show()

plt.plot(hist.history['accuracy']) 
plt.plot(hist.history['val_accuracy']) 
plt.title('Model accuracy') 
plt.ylabel('Accuracy') 
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right') 
plt.show()

In [ ]:
# Evaluate model on test set
model.evaluate(x_test, y_test_one_hot)[1]

In [ ]:
y_pred = model.predict(x_test).argmax(axis=1).argmax(axis=1)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_labels))

print(confusion_matrix(y_test, y_pred))